Upload needed installations

In [ ]:
!pip install telethon


In [ ]:
!pip install nest_asyncio


Download all media from telegram group chat

In [ ]:
import os
import asyncio
import nest_asyncio

from telethon import TelegramClient, errors
from telethon.sessions import StringSession
from telethon.tl.functions.messages import ImportChatInviteRequest

# Jupyter’daki mevcut asyncio döngüsünü “sarmak” için
nest_asyncio.apply()

# 1) API bilgilerin
api_id   = your_api_id
api_hash = 'your_api_hash'

# 2) Davet linki ve içindeki hash
invite_link = 'group_invitation_link'
invite_hash = invite_link.rsplit('+', 1)[-1]

# 3) Medyaların kaydedileceği klasör
download_folder = 'downloaded_media'

async def main():
    # Bellek içi (in-memory) session kullanıyoruz;
    # böylece .session dosyası kilitlenmez.
    session = StringSession()
    client = TelegramClient(session, api_id, api_hash)
    await client.start()

    # Gruba katılmayı dene (zaten üyeysen UserAlreadyParticipant atla)
    try:
        await client(ImportChatInviteRequest(invite_hash))
        print("Gruba katıldık.")
    except errors.UserAlreadyParticipantError:
        print("Zaten grup üyesisiniz.")
    except Exception as e:
        print("Katılırken hata:", e)

    # Sohbeti entity olarak almayı dene
    entity = None
    try:
        entity = await client.get_entity(invite_link)
        print("Entity davet linkiyle alındı:", getattr(entity, 'title', None))
    except Exception:
        # Fallback: dialogs içinde hash aranacak
        async for dialog in client.iter_dialogs():
            name     = dialog.name or ''
            username = getattr(dialog.entity, 'username', '') or ''
            if invite_hash.lower() in name.lower() or invite_hash.lower() in username.lower():
                entity = dialog.entity
                print("Dialog listesinden bulundu:", dialog.name)
                break

    if not entity:
        print("Sohbet bulunamadı. Dialog’ları kontrol et.")
        await client.disconnect()
        return

    # İndirilecek medya klasörünü hazırla
    os.makedirs(download_folder, exist_ok=True)

    # Tüm mesajları tara, medya varsa indir
    async for msg in client.iter_messages(entity, limit=None):
        if msg.media:
            ts    = msg.date.strftime('%Y%m%d_%H%M%S')
            fname = f"{msg.id}_{ts}"
            path  = await client.download_media(msg.media, file=os.path.join(download_folder, fname))
            print("İndirildi:", path)

    # İstersen bu satırla session string’ini saklayabilirsin:
    print("Session string (kaydetmek istersen):", session.save())

    await client.disconnect()
    print("Tamamlandı.")

# Jupyter içindeysen:
await main()

# Eğer bu kodu standalone .py dosyası olarak çalıştıracaksan:
# 1) nest_asyncio.apply() ve await yerine asyncio.run(main()) kullanabilirsin.


Download all media from normal chat with username or id

In [ ]:
import os
import asyncio
import nest_asyncio

from telethon import TelegramClient
from telethon.sessions import StringSession

# Jupyter’daki mevcut asyncio döngüsünü “sarmak” için
nest_asyncio.apply()

# 1) API bilgilerin
api_id   = YOUR_API_ID
api_hash = 'YOUR_API_HASH'

# 2) Sohbet için username veya chat ID
chat_target = 'telegram_chat_username_or_id'

# 3) Medyaların kaydedileceği klasör
download_folder = 'downloaded_media'

async def main():
    # Bellek içi (in-memory) session kullanıyoruz
    session = StringSession()
    client = TelegramClient(session, api_id, api_hash)
    await client.start()

    # Sohbet entity'sini al
    try:
        entity = await client.get_entity(chat_target)
        title = getattr(entity, 'title', getattr(entity, 'username', str(entity.id)))
        print(f"Sohbet bulundu: {title}")
    except Exception as e:
        print("Sohbeti alırken hata:", e)
        await client.disconnect()
        return

    # İndirilecek medya klasörünü hazırla
    os.makedirs(download_folder, exist_ok=True)

    # Tüm mesajları tara, medya varsa indir
    async for msg in client.iter_messages(entity, limit=None):
        if msg.media:
            ts    = msg.date.strftime('%Y%m%d_%H%M%S') if msg.date else ''
            fname = f"{msg.id}_{ts}"
            path  = await client.download_media(msg.media, file=os.path.join(download_folder, fname))
            print("İndirildi:", path)

    # İstersen session string’ini kaydedebilirsin
    print("Session string (kaydetmek istersen):", session.save())

    await client.disconnect()
    print("Tamamlandı.")

# ——— Jupyter Notebooks için ———
# burayı hücreye yapıştırıp çalıştır:
# await main()

# ——— Standalone .py dosyası için ———
if __name__ == '__main__':
    asyncio.run(main())
